# ranger - authorization


## [git - apache-ranger-docker-poc](https://github.com/kadensungbincho/apache-ranger-docker-poc)

- [medium](https://medium.com/swlh/hands-on-apache-ranger-docker-poc-with-hadoop-hdfs-hive-presto-814344a03a17)

<table>
    <td>
        <img src='https://miro.medium.com/max/700/1*muPN4qF8Py8ifOJDnactoA.png' width='600'>
        <p style="vertical-align: text-top; text-align: left; font-family: 'objektiv-mk1'; white-space:pre-wrap; word-wrap:break-word;">The points that I want to make on the above diagram are thickened 3:

- Plugin: 
        Every service that uses the Apache Ranger has an agent 
        specialized for each case(here they are HDFS, Hive, Presto)
            
- Audit Logs to ES: 
        audit logs are sent to the ES 
        directly from the agent(plugin) to the audit store(ES in this case)
            
- Policy Checking through Ranger Security Admin and [applying](https://github.com/apache/ranger/blob/release-ranger-2.1.0/agents-common/src/main/java/org/apache/ranger/plugin/service/RangerBasePlugin.java#L222): 
        the agents validate whether the user’s request to the resource is authorized 
        through checking the policies on the Ranger Security Admin
        (sometimes it caches — need more research on it).
</p>
    </td>
</table>

note. <font color='grey' size='5px'>this docker does NOT include spark</font>

<p style="vertical-align: text-top; text-align: left; font-family: 'objektiv-mk1,Helvetica,Arial,sans-serif,Linotte Light'; white-space:pre-wrap; word-wrap:break-word;"><font color='#00CC66'>docker-compose up -d
-> to combine multiple docker images</font></p>

```
docker network create ranger-env

cd docker-composes/ranger
docker-compose up -d --build

cd ../hadoop
docker-compose up -d --build
```



## [git - docker-hadoop-spark](https://github.com/Marcel-Jan/docker-hadoop-spark)

note. <font color='grey' size='5px'>this docker does NOT include ranger</font>

---

<p style="vertical-align: text-top; text-align: left; font-family: 'objektiv-mk1,Helvetica,Arial,sans-serif,Linotte Light'; white-space:pre-wrap; word-wrap:break-word;">
<font color='#00CC66' size='4px'>Thus, we combine these two git resources by adding spark configuration (from git 2) to hadoop configuration in docker-compose file (from git 1),
and then we $docker-compose up -d for both 'ranger' and 'hadoop' from git 1 only</font>
<font color='red'>notice.
the two configs are from different sources, we need to pay attention to those in duplicates,
for example, port of presto from git 1 is the same as port of spark master from git 2
set them differently to prevent collision</font>
</p>

---

## [git - submarine - spark plugin for ranger](https://github.com/apache/submarine)

- [download](https://submarine.apache.org/docs/download/)

Q: in this ranger docker container, where is hive-site.xml?

- A: some useful commands to find files:

    - [linux - find command](https://blog.miniasp.com/post/2010/08/27/Linux-find-command-tips-and-notice)
    
    > find /YOUR/DIR/TO/SEARCH -type f -iname '\*hive\*'
    
    > find /YOUR/DIR/TO/SEARCH -type d -iname 'YOUR_DIR_PATTERN'
    
    - [linux - grep command](https://stackoverflow.com/questions/16956810/how-do-i-find-all-files-containing-specific-text-on-linux)
    
    > grep -rnwl '/YOUR/DIR/TO/SEARCH' -e 'YOUR_PATTERN'
    
    
```
-r or -R is recursive,
-n is line number, and
-w stands for match the whole word.
-l (lower-case L) can be added to just give the file name of matching files.
-e is the pattern used during the search
```

# we need various plugins

- issues:
  - <font color='red' size='3px'>those corresponding plugins are probably not visible / not showed in ranger admin UI console</font>
  - referece for possible solutions:
      - [知乎 - 你們的大數據集群還在裸奔嗎？你需要一個Apache Ranger](https://zhuanlan.zhihu.com/p/370263573)
      - google: make "hive" plugin "visible" on "ranger" admin console
          - [Making the Hive plugin for Ranger visible](https://docs.cloudera.com/cdp-private-cloud-upgrade/latest/upgrade-cdh/topics/hive-plugin-problem.html)
          - [Apache Ranger : Issues in Syncing the UI policy to beeline and setting up of metastore plugin](https://community.cloudera.com/t5/Support-Questions/Apache-Ranger-Issues-in-Syncing-the-UI-policy-to-beeline-and/td-p/124456)
          - [cwiki - Design Proposal for Hive Metastore Plugin](https://cwiki.apache.org/confluence/display/RANGER/Design+Proposal+for+Hive+Metastore+Plugin)
          - ### [Non-Ambari Cluster Installation Guide](https://docs.cloudera.com/HDPDocuments/HDP2/HDP-2.4.2/bk_installing_manually_book/content/installing_ranger_plugins.html)
          - [Ranger Hive repository test connection fails in kerberized cluster](https://community.cloudera.com/t5/Support-Questions/Ranger-Hive-repository-test-connection-fails-in-kerberized/m-p/149087)
          - [security-workshops/Setup-ranger-22.md](https://github.com/abajwa-hw/security-workshops/blob/master/Setup-ranger-22.md#setup-hive-repo-in-ranger)
          - google: ranger docker where is hive-site.xml
              - [apiary-metastore-docker/Dockerfile](https://github.com/ExpediaGroup/apiary-metastore-docker/blob/master/Dockerfile)
          
          - [Apache Ranger - Frequently Asked Questions](https://ranger.apache.org/faq.html)
            > Does the Apache Ranger plugin need to be implemented in each datanode ? -> No, the Apache Ranger plugin for Hadoop is only needed in the NameNode.
          - [How to Configure Hive Authorization Using Apache Ranger](https://www.thegeekdiary.com/how-to-configure-hive-authorization-using-apache-ranger/)
          - [cwiki - Ranger Installation Guide](https://cwiki.apache.org/confluence/display/RANGER/Ranger+Installation+Guide)
  
- plugins
  - hive plugin
  - hdfs plugin
  - spark plugin
  - yarn plugin